# Processing Raw ACS Data

## Dependencies

In [ ]:
pip install geopandas fiona

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import fiona
import glob
from tqdm import tqdm

## Finding Metadata

In [ ]:
#Download to examples of datasets in the geodatabase from the oldest and newest years
tables12=fiona.listlayers('raw data/ACS_2012/ACS_2012_5YR_TRACT_01_ALABAMA.gdb')
tables19=fiona.listlayers('raw data/ACS_2019/ACS_2019_5YR_TRACT_01_ALABAMA.gdb')

In [ ]:
# This gives us the number of the metadata dataframe in the 2012 gdb
for i in range(len(tables12)):
    if tables12[i]=='TRACT_METADATA_2012':
        layer_metadata=i

In [ ]:
# This dataframe will tell us the metadata, such as the names of each column and what they mean
df=gpd.read_file(
    'raw data/ACS_2012/ACS_2012_5YR_TRACT_01_ALABAMA.gdb',
    layer=layer_metadata,
    ignore_geometry=True
    )

In [ ]:
df

,Short_Name,Full_Name
0,B00001e1,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: To...
1,B00001m1,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: To...
2,B00002e1,UNWEIGHTED SAMPLE HOUSING UNITS: Total: Hou...
3,B00002m1,UNWEIGHTED SAMPLE HOUSING UNITS: Total: Hou...
4,B01001e1,Total: Total Population -- (Estimate)
...,...,...
24307,B27022m13,Female: Not enrolled in school: Civilian no...
24308,B27022e14,Female: Not enrolled in school: With health ...
24309,B27022m14,Female: Not enrolled in school: With health ...
24310,B27022e15,Female: Not enrolled in school: No health in...


 **Variables to include based on scouring metadata**

 Total_pop, male_pop, female_pop  
 B01001e1, B01001e2, B01001e26

 Column to calculate total number under 18  
 B09001e1

 Columns to calculate elderly  
 B09020e1

 Median ages  
 B01002e1, B01002e2, B01002e3

 Race alone  
 B02001e2, B02001e3, B02001e4, B02001e5, B02001e6, B02001e7, B02001e8, B02005e1

 Ethnicity and Race (Hispanic)  
 B03003e3, B03002e3, B03002e4, B03002e5, B03002e6, B03002e7, B03002e8, B03002e9
 
 Citizenship Status, foreign born, born in state, born out of state  
 B05001e6, B05002e13, B05002e3, B05002e4

Population over 5/Does not speak English well  
 B06007e1, B06007e5, B06007e8

 Workers, worked at home, commute time, no car  
 B08006e1, B08006e17, B08013e1, B08014e2

 Women of childbearing age  
 B13001e1

 Educational Attainment  
 B15003m1, B15003e17, B15003e18, B15003e19, B15003e20, B15003e21, B15003e22,
 B15003e23, B15003e24, B15003e25

 Poverty  
 B17001e1,B17001e2

 Disability  
 B18135e13, B18135e14

 Income  
 B19001e1, B19001e2, B19001e3, B19001e4, B19001e5, B19001e6, B19001e7, B19001e8,
 B19001e9, B19001e10, B19001e11, B19001e12, B19001e13, B19001e14, B19001e15,
 B19001e16, B19001e17, B19013e1

 Food Stamps  
 B22001e1, B22001e2

 Job Status  
 B23025e1, B23025e2, B23025e3, B23025e4, B23025e5, B23025e6, B23025e7

Job type  
 B24011e1, B24011e2, B24011e18, B24011e26, B24011e29, B24011e33

Housing  
 B25001e1, B25002e2, B25002e3, B25003e2, B25003e3, B25077e1, B25076e1, B25078e1,
 B25111e1

In [ ]:
# Create a dataframe of just the metadata we will be using
columns=['B01001e1', 'B01001e2', 'B01001e26', 'B09001e1', 'B09020e1','B01002e1', 'B01002e2', 'B01002e3',
         'B02001e2', 'B02001e3', 'B02001e4', 'B02001e5', 'B02001e6', 'B02001e7', 'B02001e8', 'B02005e1',
         'B03003e3', 'B03002e3', 'B03002e4', 'B03002e5', 'B03002e6', 'B03002e7', 'B03002e8', 'B03002e9',
         'B05001e6', 'B05002e13', 'B05002e3', 'B05002e4', 'B06007e1', 'B06007e5', 'B06007e8',
         'B08006e1', 'B08006e17', 'B08013e1', 'B08014e2', 'B13001e1', 'B15003e1', 'B15003e17',
         'B15003e18', 'B15003e19', 'B15003e20', 'B15003e21', 'B15003e22', 'B15003e23', 'B15003e24',
         'B15003e25', 'B17001e1', 'B17001e2', 'B18135e13', 'B18135e14', 'B19001e1', 'B19001e2',
         'B19001e3', 'B19001e4', 'B19001e5', 'B19001e6', 'B19001e7', 'B19001e8', 'B19001e9', 'B19001e10',
         'B19001e11', 'B19001e12', 'B19001e13', 'B19001e14', 'B19001e15', 'B19001e16', 'B19001e17',
         'B19013e1', 'B22001e1', 'B22001e2', 'B23025e1', 'B23025e2', 'B23025e3', 'B23025e4', 'B23025e5',
         'B23025e6', 'B23025e7', 'B24011e1', 'B24011e2', 'B24011e18', 'B24011e26', 'B24011e29',
         'B24011e33', 'B25001e1', 'B25002e2', 'B25002e3', 'B25003e2', 'B25003e3', 'B25077e1',
         'B25076e1', 'B25078e1', 'B25111e1']
meta_dict=df.loc[df.Short_Name.isin(columns)]
meta_dict=meta_dict.reset_index()[['Short_Name','Full_Name']]

In [ ]:
#API's rely on a slightly different naming convention to operate. We will apply that column to the data too.
api_col=['B01001_001E', 'B01001_002E', 'B01001_026E', 'B09001_001E', 'B09020_001E', 'B01002_001E', 'B01002_002E', 'B01002_003E',
         'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E', 'B02001_007E', 'B02001_008E', 'B02005_001E',
         'B03003_003E', 'B03002_003E', 'B03002_004E', 'B03002_005E', 'B03002_006E', 'B03002_007E', 'B03002_008E', 'B03002_009E',
         'B05001_006E', 'B05002_013E', 'B05002_003E', 'B05002_004E', 'B06007_001E', 'B06007_005E', 'B06007_008E', 'B08006_001E',
         'B08006_017E', 'B08013_001E', 'B08014_002E', 'B13001_001E', 'B15003_001E', 'B15003_017E', 'B15003_018E', 'B15003_019E',
         'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E', 'B17001_001E', 'B17001_002E',
         'B18135_013E', 'B18135_014E', 'B19001_001E', 'B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E',
         'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E', 'B19001_012E', 'B19001_013E', 'B19001_014E',
         'B19001_015E', 'B19001_016E', 'B19001_017E', 'B19013_001E', 'B22001_001E', 'B22001_002E', 'B23025_001E', 'B23025_002E',
         'B23025_003E', 'B23025_004E', 'B23025_005E', 'B23025_006E', 'B23025_007E', 'B24011_001E', 'B24011_002E', 'B24011_018E',
         'B24011_026E', 'B24011_029E', 'B24011_033E', 'B25001_001E', 'B25002_002E', 'B25002_003E', 'B25003_002E', 'B25003_003E',
         'B25077_001E', 'B25076_001E', 'B25078_001E', 'B25111_001E']

meta_dict['API_Name']=api_col

In [ ]:
#Sanity check to ensure everything makes sense
meta_dict=meta_dict[['API_Name','Short_Name','Full_Name']]
meta_dict.head()

,API_Name,Short_Name,Full_Name
0,B01001_001E,B01001e1,Total: Total Population -- (Estimate)
1,B01001_002E,B01001e2,Male: Total Population -- (Estimate)
2,B01001_026E,B01001e26,Female: Total Population -- (Estimate)
3,B09001_001E,B01002e1,Median age: Total: Total population -- (Est...
4,B09020_001E,B01002e2,Median age: Male: Total population -- (Esti...


In [ ]:
# Export to save a copy
meta_dict.to_csv('raw data/metadata_dictionary.csv')

## List the states with their number code

In [ ]:
# lists all of the names of the sheets in ACS_2012
import glob
file12=glob.glob("raw data/ACS_2012/*")

In [ ]:
# This returns just the names and numbers of each state in the file- we can use this in our loop below
exten=[]
for i in range(len(file12)):
  exten.append(file12[i][37:-4])
exten

['55_WISCONSIN',
 '56_WYOMING',
 '72_PUERTO_RICO',
 '01_ALABAMA',
 '02_ALASKA',
 '04_ARIZONA',
 '05_ARKANSAS',
 '06_CALIFORNIA',
 '08_COLORADO',
 '09_CONNECTICUT',
 '10_DELAWARE',
 '11_DISTRICT_OF_COLUMBIA',
 '12_FLORIDA',
 '13_GEORGIA',
 '15_HAWAII',
 '16_IDAHO',
 '17_ILLINOIS',
 '18_INDIANA',
 '19_IOWA',
 '20_KANSAS',
 '21_KENTUCKY',
 '22_LOUISIANA',
 '23_MAINE',
 '24_MARYLAND',
 '25_MASSACHUSETTS',
 '26_MICHIGAN',
 '27_MINNESOTA',
 '28_MISSISSIPPI',
 '29_MISSOURI',
 '30_MONTANA',
 '31_NEBRASKA',
 '32_NEVADA',
 '33_NEW_HAMPSHIRE',
 '34_NEW_JERSEY',
 '35_NEW_MEXICO',
 '36_NEW_YORK',
 '37_NORTH_CAROLINA',
 '38_NORTH_DAKOTA',
 '39_OHIO',
 '40_OKLAHOMA',
 '41_OREGON',
 '42_PENNSYLVANIA',
 '44_RHODE_ISLAND',
 '45_SOUTH_CAROLINA',
 '46_SOUTH_DAKOTA',
 '47_TENNESSEE',
 '48_TEXAS',
 '49_UTAH',
 '50_VERMONT',
 '51_VIRGINIA',
 '53_WASHINGTON',
 '54_WEST_VIRGINIA']

## read in tracts geometry

In [ ]:
geo = gpd.read_file('raw data/all_tracts_2019/all_tracts_2019.shp')

## functions to process data



In [78]:
def stitch_dataframes(year):
  '''
  takes year. combines Census data tables for all states for that year. returns dataframe
  '''
  
  combo_lists=[]
  for state in tqdm(exten):
      path = f'raw data/ACS_{year}/ACS_{year}_5YR_TRACT_{state}.gdb'
      
      df01=gpd.read_file(path,layer='X01_AGE_AND_SEX', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[0:6]['Short_Name'])]
      df02=gpd.read_file(path,layer='X02_RACE', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[6:13]['Short_Name'])]
      df03=gpd.read_file(path,layer='X03_HISPANIC_OR_LATINO_ORIGIN', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[14:22]['Short_Name'])]
      df05=gpd.read_file(path,layer='X05_FOREIGN_BORN_CITIZENSHIP', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[22:26]['Short_Name'])]
      df06=gpd.read_file(path,layer='X06_PLACE_OF_BIRTH', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[26:29]['Short_Name'])]
      df08=gpd.read_file(path,layer='X08_COMMUTING', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[29:33]['Short_Name'])]
      df09=gpd.read_file(path,layer='X09_CHILDREN_HOUSEHOLD_RELATIONSHIP', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[33:35]['Short_Name'])]
      df15=gpd.read_file(path,layer='X15_EDUCATIONAL_ATTAINMENT', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[36:46]['Short_Name'])]
      df17=gpd.read_file(path,layer='X17_POVERTY', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[46:48]['Short_Name'])]
      df18=gpd.read_file(path,layer='X18_DISABILITY', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[48:50]['Short_Name'])]
      df19=gpd.read_file(path,layer='X19_INCOME', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[50:68]['Short_Name'])]
      df22=gpd.read_file(path,layer='X22_FOOD_STAMPS', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[68:70]['Short_Name'])]
      df23=gpd.read_file(path,layer='X23_EMPLOYMENT_STATUS', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[70:77]['Short_Name'])]
      df24=gpd.read_file(path,layer='X24_INDUSTRY_OCCUPATION', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[77:83]['Short_Name'])]
      df25=gpd.read_file(path,layer='X25_HOUSING_CHARACTERISTICS', ignore_geometry=True)[['GEOID'] + list(meta_dict.iloc[83:]['Short_Name'])]

      df_combo=df01.merge(df02,on='GEOID')
      df_combo=df_combo.merge(df03,on='GEOID')
      df_combo=df_combo.merge(df05,on='GEOID')
      df_combo=df_combo.merge(df06,on='GEOID')
      df_combo=df_combo.merge(df08,on='GEOID')
      df_combo=df_combo.merge(df09,on='GEOID')
      df_combo=df_combo.merge(df15,on='GEOID')
      df_combo=df_combo.merge(df17,on='GEOID')
      df_combo=df_combo.merge(df18,on='GEOID')
      df_combo=df_combo.merge(df19,on='GEOID')
      df_combo=df_combo.merge(df22,on='GEOID')
      df_combo=df_combo.merge(df23,on='GEOID')
      df_combo=df_combo.merge(df24,on='GEOID')
      df_combo=df_combo.merge(df25,on='GEOID')
      combo_lists.append(df_combo)


  acs_year = pd.concat(combo_lists)

  # force numeric datatype for all data columns
  acs_year = acs_year.set_index('GEOID').apply(pd.to_numeric, errors='coerce').reset_index()


  return acs_year

In [77]:
def add_geom(acs_year):
  '''
  takes combined Census data DataFrame. appends Tract geometry. returns GeoDataFrame
  '''

  dataframe = None
  dataframe = acs_year.copy()

  dataframe['GEOID'] = dataframe['GEOID'].str.slice(start=7)
  merged = geo.merge(dataframe,on='GEOID', how='inner')
  
  return merged

In [76]:
def compute_rates(acs_year_geom):
  '''
  Takes combined Census GeoDataFrame (with Tract gemoetry). Computes rates. Fills NaNs. Returns GeoDataFrame
  '''

  df = None
  df = acs_year_geom.copy()

  # drop rows with 0 population
  df = df[df['B01001e1'] > 0]

  # density
  df['density']=df['B01001e1']/df['ALAND']

  # Percent Male and Female
  df['percent_male']=df['B01001e2']/df['B01001e1']
  df['percent_female']=df['B01001e26']/df['B01001e1']

  # Racial Demographics (percentage of total population)
  df['white']=(df['B02001e2']/df['B01001e1'])
  df['black']=(df['B02001e3']/df['B01001e1'])
  df['native']=(df['B02001e4']/df['B01001e1'])
  df['asian']=(df['B02001e5']/df['B01001e1'])
  df['pacific_islander']=(df['B02001e6']/df['B01001e1'])
  df['other']=(df['B02001e7']/df['B01001e1'])
  df['mixed']=(df['B02001e8']/df['B01001e1'])

  # Ethnic Demographics (percentage of total population)
  df['white_nonhispanic']=(df['B03002e3']/df['B01001e1'])
  df['black_nonhispanic']=(df['B03002e4']/df['B01001e1'])
  df['native_nonhispanic']=(df['B03002e5']/df['B01001e1'])
  df['asian_nonhispanic']=(df['B03002e6']/df['B01001e1'])
  df['pacific_islander_nonhispanic']=(df['B03002e7']/df['B01001e1'])
  df['other_nonhispanic']=(df['B03002e8']/df['B01001e1'])
  df['mixed_nonhispanic']=(df['B03002e9']/df['B01001e1'])
  df['hispanic']=(df['B03003e3']/df['B01001e1'])

  # Percent non-citizens
  df['percent_noncitizen']=(df['B05001e6']/df['B01001e1'])

  # Place of Birth
  df['born_in_state']=df['B05002e3']/df['B01001e1']
  df['born_in_other_state']=df['B05002e4']/df['B01001e1']
  df['foreign_born']=df['B05002e13']/df['B01001e1']

  # Language Competancy
  df['spanish_no_english']=df['B06007e5']/df['B06007e1']
  df['no_english']=df['B06007e8']/df['B06007e1']

  # Age Groups
  df['under_18']=(df['B09001e1']/df['B01001e1'])
  df['over_18']=(df['B09020e1']/df['B01001e1'])

  # Education (percent of population over 25)
  df['highschool_or_less']=((df['B15003e17']+df['B15003e18'])/df['B15003e1'])
  df['no_bachelor_degree']=((df['B15003e17']+df['B15003e18']+
                              df['B15003e19']+df['B15003e20']+
                              df['B15003e21'])/df['B15003e1'])
  df['bachelor_degree_only']=(df['B15003e22']/df['B15003e1'])
  df['advanced_degree']=((df['B15003e23']+df['B15003e24']+
                              df['B15003e25'])/df['B15003e1'])
  
  # Disability (percent of noninstitutionalized population 18-64)
  df['percent_disabled']=(df['B18135e14']/df['B18135e13'])

  # Work statistics (percent of workers over 16)
  df['work_from_home'] = (df['B08006e17']/df['B08006e1'])
  df['average_commute'] = df['B08013e1']/(df['B08006e1']-df['B08006e17'])
  median_commute = df.average_commute.median()
  df.loc[(df['B08006e1'] - df['B08006e17'] <= 0), 'average_commute'] = 0
  df['average_commute'] = df['average_commute'].fillna(median_commute)

  # Poverty (percent of population for which status is determinable)
  df['percent_poverty']=(df['B17001e2']/df['B17001e1'])

  # Household income (percent of houselholds)
  df['less_than_10k']=(df['B19001e2']/df['B19001e1'])
  df['10k_15k']=(df['B19001e3']/df['B19001e1'])
  df['15k_20k']=(df['B19001e4']/df['B19001e1'])
  df['20k_25k']=(df['B19001e5']/df['B19001e1'])
  df['25k_30k']=(df['B19001e6']/df['B19001e1'])
  df['30k_35k']=(df['B19001e7']/df['B19001e1'])
  df['35k_40k']=(df['B19001e8']/df['B19001e1'])
  df['40k_45k']=(df['B19001e9']/df['B19001e1'])
  df['45k_50k']=(df['B19001e10']/df['B19001e1'])
  df['50k_60k']=(df['B19001e11']/df['B19001e1'])
  df['60k_75k']=(df['B19001e12']/df['B19001e1'])
  df['75k_100k']=(df['B19001e13']/df['B19001e1'])
  df['100k_125k']=(df['B19001e14']/df['B19001e1'])
  df['125k_150k']=(df['B19001e15']/df['B19001e1'])
  df['150k_200k']=(df['B19001e16']/df['B19001e1'])
  df['more_than_200k']=(df['B19001e17']/df['B19001e1'])

  # Receiving Food Stamps (percentage of households)
  df['percentage_food_stamps']=(df['B22001e1']/df['B22001e1'])

  # Employment statistics
  df['percentage_employed']=(df['B23025e4']/df['B23025e1'])
  df['percentage_unemployed']=(df['B23025e5']/df['B23025e1'])
  df['civilian_labor_force_percentage']=(df['B23025e3']/df['B23025e1'])
  df['percentage_in_military']=(df['B23025e6']/df['B23025e1'])
  df['percentage_outside_labor_force']=(df['B23025e7']/df['B23025e1'])

  # Professions (as percent of employed population)
  df['management_business_science_art']=df['B24011e2']/df['B24011e1']
  df['service_professions']=df['B24011e18']/df['B24011e1']
  df['sales_office_professions']=df['B24011e26']/df['B24011e1']
  df['naturalresources_construction_maintenance']=df['B24011e29']/df['B24011e1']
  df['production_transportation_professions']=df['B24011e33']/df['B24011e1']

  # Housing
  df['occupied']=(df['B25002e2']/df['B25001e1'])
  df['vacant']=(df['B25002e3']/df['B25001e1'])
  df['owner_occupied']=(df['B25003e2']/df['B25001e1'])
  df['rented']=(df['B25003e3']/df['B25001e1'])


  # rename columns
  column_dict={'B01001e1':'population','B01002e1':'median_age',
             'B01002e2':'median_age_male','B01002e3':'median_age_female',
             'B19013e1':'median_household_income',
             'B25076e1':'home_value_low_quartile','B25077e1':'home_value_median',
             'B25078e1':'home_value_hi_quartile','B25111e1':'median_rent'}
  df=df.rename(columns=column_dict)

  # drop columns
  keep_cols = ['population','density','median_age','median_age_male','median_age_female',
       'percent_male','percent_female', 'white', 'black', 'native', 'asian',
       'pacific_islander', 'other', 'mixed', 'white_nonhispanic',
       'black_nonhispanic', 'native_nonhispanic', 'asian_nonhispanic',
       'pacific_islander_nonhispanic', 'other_nonhispanic',
       'mixed_nonhispanic', 'hispanic', 'percent_noncitizen', 'born_in_state',
       'born_in_other_state', 'foreign_born', 'spanish_no_english',
       'no_english', 'work_from_home', 'average_commute', 'under_18',
       'over_18', 'highschool_or_less',
       'no_bachelor_degree', 'bachelor_degree_only', 'advanced_degree',
       'percent_poverty', 'percent_disabled', 'less_than_10k', '10k_15k',
       '15k_20k', '20k_25k', '25k_30k', '30k_35k', '35k_40k', '40k_45k',
       '45k_50k', '50k_60k', '60k_75k', '75k_100k', '100k_125k', '125k_150k',
       '150k_200k', 'more_than_200k', 'percentage_food_stamps',
       'percentage_employed', 'percentage_unemployed',
       'civilian_labor_force_percentage', 'percentage_in_military',
       'percentage_outside_labor_force', 'management_business_science_art',
       'service_professions', 'sales_office_professions',
       'naturalresources_construction_maintenance',
       'production_transportation_professions', 'occupied', 'vacant',
       'owner_occupied', 'rented','median_household_income',
       'home_value_low_quartile','home_value_median','home_value_hi_quartile',
       'median_rent','STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'ALAND', 
       'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'
       ]

  df=df[keep_cols]


  ## fill NaN from county or state median
  county_median = df.groupby(by=['STATEFP','COUNTYFP'],as_index=False).agg({
      'population':'median','density':'median','median_age':'median','median_age_male':'median','median_age_female':'median',
      'percent_male':'median','percent_female':'median','white':'median','black':'median',
      'native':'median', 'asian':'median','pacific_islander':'median','other':'median',
      'mixed':'median','white_nonhispanic':'median','black_nonhispanic':'median',
      'native_nonhispanic':'median','asian_nonhispanic':'median','pacific_islander_nonhispanic':'median',
      'other_nonhispanic':'median','mixed_nonhispanic':'median','hispanic':'median',
      'percent_noncitizen':'median','born_in_state':'median','born_in_other_state':'median',
      'foreign_born':'median','spanish_no_english':'median','no_english':'median',
      'work_from_home':'median','average_commute':'median','under_18':'median','over_18':'median',
      'highschool_or_less':'median','no_bachelor_degree':'median',
      'bachelor_degree_only':'median','advanced_degree':'median','percent_poverty':'median',
      'percent_disabled':'median','less_than_10k':'median','10k_15k':'median','15k_20k':'median',
      '20k_25k':'median','25k_30k':'median','30k_35k':'median','35k_40k':'median','40k_45k':'median',
      '45k_50k':'median','50k_60k':'median','60k_75k':'median','75k_100k':'median','100k_125k':'median',
      '125k_150k':'median','150k_200k':'median','more_than_200k':'median',
      'percentage_food_stamps':'median','percentage_employed':'median','percentage_unemployed':'median',
      'civilian_labor_force_percentage':'median','percentage_in_military':'median',
      'percentage_outside_labor_force':'median','management_business_science_art':'median',
      'service_professions':'median','sales_office_professions':'median',
      'naturalresources_construction_maintenance':'median','production_transportation_professions':'median',
      'occupied':'median','vacant':'median','owner_occupied':'median','rented':'median',
      'median_household_income':'median','home_value_low_quartile':'median','home_value_median':'median',
      'home_value_hi_quartile':'median', 'median_rent':'median'})
  
  county_median = pd.DataFrame(
    index=df.index,
    data=df[['STATEFP','COUNTYFP']]
  ).merge(
    county_median,
    on=['STATEFP','COUNTYFP'],
    how='left'
  )

  state_median=df.groupby(by=['STATEFP'],as_index=False).agg({'population':'median',
    'median_age':'median','density':'median','median_age_male':'median','median_age_female':'median',
    'percent_male':'median','percent_female':'median','white':'median','black':'median',
    'native':'median', 'asian':'median','pacific_islander':'median','other':'median',
    'mixed':'median','white_nonhispanic':'median','black_nonhispanic':'median',
    'native_nonhispanic':'median','asian_nonhispanic':'median','pacific_islander_nonhispanic':'median',
    'other_nonhispanic':'median','mixed_nonhispanic':'median','hispanic':'median',
    'percent_noncitizen':'median','born_in_state':'median','born_in_other_state':'median',
    'foreign_born':'median','spanish_no_english':'median','no_english':'median',
    'work_from_home':'median','average_commute':'median','under_18':'median','over_18':'median',
    'highschool_or_less':'median','no_bachelor_degree':'median',
    'bachelor_degree_only':'median','advanced_degree':'median','percent_poverty':'median',
    'percent_disabled':'median','less_than_10k':'median','10k_15k':'median','15k_20k':'median',
    '20k_25k':'median','25k_30k':'median','30k_35k':'median','35k_40k':'median','40k_45k':'median',
    '45k_50k':'median','50k_60k':'median','60k_75k':'median','75k_100k':'median','100k_125k':'median',
    '125k_150k':'median','150k_200k':'median','more_than_200k':'median',
    'percentage_food_stamps':'median','percentage_employed':'median','percentage_unemployed':'median',
    'civilian_labor_force_percentage':'median','percentage_in_military':'median',
    'percentage_outside_labor_force':'median','management_business_science_art':'median',
    'service_professions':'median','sales_office_professions':'median',
    'naturalresources_construction_maintenance':'median','production_transportation_professions':'median',
    'occupied':'median','vacant':'median','owner_occupied':'median','rented':'median',
    'median_household_income':'median','home_value_low_quartile':'median','home_value_median':'median',
    'home_value_hi_quartile':'median', 'median_rent':'median'})
  
  state_median = pd.DataFrame(
    index=df.index,
    data=df[['STATEFP']]
  ).merge(
    state_median,
    on='STATEFP',
    how='left'
  )

  df = df.fillna(
    county_median
  ).fillna(
    state_median
  ).dropna()

  return df


## process all years

In [ ]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [ ]:
for year in years:
  print(f'__starting {year}__')
  
  acs_year = stitch_dataframes(year)
  acs_year_geo = add_geom(acs_year)
  acs_year_rates = compute_rates(acs_year_geo)

  year_slug = str(year)[2:]
  acs_year_rates.to_csv(f'processed data/acs_{year_slug}_filled.csv')
  
  print(f'*saved: {year}*')